In [1]:
%%javascript
require.config({
    paths: { 
        d3: 'https://d3js.org/d3.v5.min'
    }
});

<IPython.core.display.Javascript object>

In [2]:
from IPython.display import display, Javascript, HTML
import json

# This is needed to load d3
display(HTML(filename='tree.css.html'));
display(Javascript(filename="tree.js"))

<IPython.core.display.Javascript object>

In [3]:
def draw_network(data, width=600, height=400):
    """Uses D3 to draw a graph
    
    Parameters:
    data: List of Lists
          Each list contains links of the form [parent_node_label, child_node_label]
          
    Is unable to draw nodes not connected to any other edges
    """
    display(Javascript("""
        (function(element){
            require(['tree'], function(tree) {
                tree(element.get(0), %s, %d, %d);
            });
        })(element);
    """ % (json.dumps(data), width, height)))
    

def draw_dag(parents_to_children, width=600, height=400):
    """Draws a DAG
    
    Parameters:
    parents_to_children: Dictionary
                         Keys are nodes, values are a list of children
    """
    network_data = []
    for parent, children in parents_to_children.items():
        links = [[parent, child] for child in children]
        network_data.extend(links)
    draw_network(network_data, width, height)

In [4]:
# Example of usage
draw_dag({'A': ['B', 'C'], 
          'B': [], 
          'C': ['D', 'E'], 
          'D': [], 
          'E': []})

<IPython.core.display.Javascript object>

## Start causal inference / data science imports

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

%matplotlib inline

Let's look at an example from daggity's [Learn Simpson's Rule](http://dagitty.net/learn/simpson/) example. This is the machine they have with 3 levels:

In [6]:
draw_dag({
    'Z1': ['Z3', 'U'],
    'Z3': ['Z5', 'Z2'],
    'Z5': ['Z4', 'Y'],
    'U': ['Z4', 'X', 'Z2'],
    'Z4': [],
    'Z2': [],
    'X': ['Y'],
    'Y': []})

<IPython.core.display.Javascript object>

Here is an example of a data generation process that follows this DAG. A variable can only by defined in terms of either a new randomly generated value (in this case, all normal distributions) _or_ from random variables that are it's parent.

e.g. 
```
Z5 = F(np.random.<something> , Z3)
```
can only happen because `Z3` is a parent of `Z5` in the DAG. We cannot have `Z5 = F(np.random.<something>, Z3, Z1)` because `Z1` is not a parent of `Z3`.

**Simplification**

Here we have assumed that all random number generation processes are normal, and all functions are linear. These are not necessary assumptions for causal reasoning.

In [7]:
def simpson_three_level_data_generation(samples, stddev, causal_effect):
    Z1 = np.random.normal(scale=stddev, size=samples)
    Z3 = np.random.normal(scale=stddev, size=samples) + Z1
    Z5 = np.random.normal(scale=stddev, size=samples) + Z3
    U  = np.random.normal(scale=stddev, size=samples) + Z1
    Z4 = np.random.normal(scale=stddev, size=samples) + Z5 + U
    Z2 = np.random.normal(scale=stddev, size=samples) + Z3 + U
    X  = np.random.normal(scale=stddev, size=samples) + U
    Y  = np.random.normal(scale=stddev, size=samples) + causal_effect*X + 10*Z5
    return pd.DataFrame({
        'X': X,
        'Z1': Z1,
        'Z2': Z2,
        'Z3': Z3,
        'Z4': Z4,
        'Z5': Z5,
        'Y': Y})

In [8]:
# Example of using regression, controlling for everything
data = simpson_three_level_data_generation(10000, 2.2, causal_effect = 2)
features = data.drop('Y', axis=1)
target = data.Y

lr = LinearRegression().fit(features, target)

dict(zip(features.columns, lr.coef_))

{'X': 1.9958860926926953,
 'Z1': 0.02243065872983781,
 'Z2': 0.001270851867811536,
 'Z3': -0.012468208455256286,
 'Z4': -0.0039012163959659374,
 'Z5': 10.008369085482956}

We haven't done too poorly detecting the causal effect between `X` and `Y` controlling for everything. Let's not control for anything:

In [9]:
features = data[['X']]
target = data.Y

lr = LinearRegression().fit(features, target)

dict(zip(features.columns, lr.coef_))

{'X': 5.304148728446097}

It maybe isn't surprising that controlling for nothing leaves all the pressure on `X`. Let's try just controlling for `Z1`:

In [10]:
features = data[['X', 'Z1']]
target = data.Y

lr = LinearRegression().fit(features, target)

dict(zip(features.columns, lr.coef_))

{'X': 2.04474052913836, 'Z1': 9.750580491492672}

Doing better. In fact, we can make a function to control for different subsets of variables:

In [11]:
def get_coefficients(df, feature_names=None, target_name='Y'):
    if feature_names is None:
        feature_names = [col for col in df.columns if col != target_name]
    features = df[feature_names]
    target = df[target_name]
    lr = LinearRegression().fit(features, target)
    coef = dict(zip(feature_names, lr.coef_))
    return pd.Series(coef).to_frame().T
        

Redoing the previous calculations, and recall that by construction the causal effect of `X` on `Y` is 2:

In [12]:
get_coefficients(data)

,X,Z1,Z2,Z3,Z4,Z5
0,1.995886,0.022431,0.001271,-0.012468,-0.003901,10.008369


In [13]:
get_coefficients(data, ['X'])

,X
0,5.304149


In [14]:
get_coefficients(data, ['X', 'Z1'])

,X,Z1
0,2.044741,9.75058


Let's look at some other conditions:

In [15]:
get_coefficients(data, ['X', 'Z1', 'Z3', 'Z5'])

,X,Z1,Z3,Z5
0,1.994564,0.02112,-0.011193,10.004513


We get a _great_ answer, simply by controlling for `Z5`

In [16]:
get_coefficients(data, ['X', 'Z5'])

,X,Z5
0,1.998516,10.002746


If we control for `Z1`, `Z2`, and `Z3` we get a good estimate:

In [17]:
get_coefficients(data, ['X', 'Z1', 'Z2', 'Z3'])

,X,Z1,Z2,Z3
0,2.143947,0.028439,-0.224261,10.23088


Note that more conditioning can make things worse. We decide to add `Z4` "just to be safe" and our estimate of the  (direct) effect of `X` on `Y` gets _worse_:

In [18]:
get_coefficients(data, ['X', 'Z1', 'Z2', 'Z3', 'Z4'])

,X,Z1,Z2,Z3,Z4
0,0.625356,-1.41728,-1.488508,7.288382,4.255306


## A more concrete example: Salary and Experience

In [19]:
N = 3000

education = np.random.choice([0, 1, 2],p=[0.05,0.75,0.2], size=(N))
experience = np.random.binomial(25, p=0.8, size=(N))
gap_years = np.random.choice([0, 1, 2],p=[0.2,0.7,0.1], size=(N))

age = 18 + 4*education + experience + gap_years

salary = 3*experience + 2*education

df = pd.DataFrame({
    'education': education, 'experience': experience, 'gap': gap_years, 'age': age, 'salary': salary
})

The corresponding DAG is

In [22]:
draw_dag({'education': ['salary', 'age'],
          'experience': ['age', 'salary'],
          'gap_years': ['age'],
          'salary': []})

<IPython.core.display.Javascript object>

By rearranging the DAG, you should be able to see that `salary` only depends on `education` and `experience` (directly), and that the other variables are not relevant. Taking our data:

In [23]:
get_coefficients(df, ['education', 'experience'], 'salary')

,education,experience
0,2.0,3.0


Naturally we get the answer exactly right! What if we (incorrectly) control for age? Does controlling for an irrelevant variable matter?

In [24]:
get_coefficients(df, ['education', 'experience', 'age'], 'salary')

,education,experience,age
0,2.0,3.0,2.410947e-16


In this case, no. What about gap year?

In [27]:
get_coefficients(df, ['education', 'experience', 'gap'], 'salary')

,education,experience,gap
0,2.0,3.0,-1.189484e-15


Great! It seems to make no difference.

What if we control for "everything" (i.e. both age and gap, rather than both separately?)

In [29]:
get_coefficients(df, ['education', 'experience', 'gap', 'age'], 'salary')

,education,experience,gap,age
0,-1.527794,2.118052,-0.881948,0.881948


Oh no!! We see that we now have a _negative_ correlation between education and salary!